In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
plt.style.use('dark_background')

from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pyspark.sql import functions as F
from pyspark.sql.types import *

23/05/02 19:10:37 WARN Utils: Your hostname, rig resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp6s0)
23/05/02 19:10:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/02 19:10:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Introduction to data pipelines

## Quick pipeline

In [2]:
# Import the data to a DataFrame
departures_df = spark.read.csv('AA_DFW_2015_Departures_Short.csv.gz', header=True)

# Remove any duration of 0
departures_df = departures_df.filter(departures_df[3] > 0)

# Add an ID column
departures_df = departures_df.withColumn('id', F.monotonically_increasing_id())

# Write the file out to JSON format
departures_df.write.json('output.json', mode='overwrite')

In [3]:
departures_df.printSchema()

root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Flight Number: string (nullable = true)
 |-- Destination Airport: string (nullable = true)
 |-- Actual elapsed time (Minutes): string (nullable = true)
 |-- id: long (nullable = false)



In [4]:
departures_df.show(3)

+-----------------+-------------+-------------------+-----------------------------+---+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)| id|
+-----------------+-------------+-------------------+-----------------------------+---+
|       01/01/2015|         0005|                HNL|                          526|  0|
|       01/01/2015|         0007|                OGG|                          517|  1|
|       01/01/2015|         0023|                SFO|                          233|  2|
+-----------------+-------------+-------------------+-----------------------------+---+
only showing top 3 rows



## Data handling techniques

## Removing commented lines

In [5]:
# Import the file to a DataFrame and perform a row count
annotations_df = spark.read.csv('annotations.csv', sep='|')
full_count = annotations_df.count()
# Count the number of rows beginning with '#'
comment_count = annotations_df.where(F.col('_c0').startswith('#')).count()
# Import the file to a new DataFrame, without commented rows
no_comments_df = spark.read.csv('annotations.csv', sep='|', comment='#')
# Count the new DataFrame and verify the difference is as expected
no_comments_count = no_comments_df.count()
print("Full count: %d\nComment count: %d\nRemaining count: %d" % (full_count, comment_count, no_comments_count))

Full count: 100
Comment count: 4
Remaining count: 96


## Removing invalid rows

In [6]:
# Split _c0 on the tab character and store the list in a variable
tmp_fields = F.split(annotations_df['_c0'], '\t')

# Create the colcount column on the DataFrame
annotations_df = annotations_df.withColumn('colcount', F.size(tmp_fields))

# Remove any rows containing fewer than 5 fields
annotations_df_filtered = annotations_df.filter(~ (F.col('colcount') < 5))

# Count the number of rows
final_count = annotations_df_filtered.count()
print("Initial count: %d\nFinal count: %d" % (no_comments_count, final_count))

Initial count: 96
Final count: 65


## Splitting into columns

In [7]:
# Split the content of _c0 on the tab character (aka, '\t')
split_cols = F.split(annotations_df_filtered['_c0'], '\t')

# Add the columns folder, filename, width, and height
split_df = annotations_df_filtered.withColumn('folder', split_cols.getItem(0))
split_df = split_df.withColumn('filename', split_cols.getItem(1))
split_df = split_df.withColumn('width', split_cols.getItem(2))
split_df = split_df.withColumn('height', split_cols.getItem(3))

# Add split_cols as a column
split_df = split_df.withColumn('split_cols', split_cols)

split_df.show(truncate=60)

+------------------------------------------------------------+--------+--------+---------------+-----+------+------------------------------------------------------------+
|                                                         _c0|colcount|  folder|       filename|width|height|                                                  split_cols|
+------------------------------------------------------------+--------+--------+---------------+-----+------+------------------------------------------------------------+
|02110627\tn02110627_12938\t200\t300\taffenpinscher,0,9,17...|       5|02110627|n02110627_12938|  200|   300|[02110627, n02110627_12938, 200, 300, affenpinscher,0,9,1...|
|02093754\tn02093754_1148\t500\t378\tBorder_terrier,73,127...|       5|02093754| n02093754_1148|  500|   378|[02093754, n02093754_1148, 500, 378, Border_terrier,73,12...|
|          %s\t%s\t800\t600\tShetland_sheepdog,124,87,576,514|       5|      %s|             %s|  800|   600|        [%s, %s, 800, 600, Shetland_

## Further parsing

In [8]:
def retriever(cols, colcount):
  # Return a list of dog data
  return cols[4:colcount]

# Define the method as a UDF
udfRetriever = F.udf(retriever, ArrayType(StringType()))

# Create a new column using your UDF
split_df = split_df.withColumn('dog_list', udfRetriever(split_df.split_cols, split_df.colcount))

# Remove the original column, split_cols, and the colcount
split_df = split_df.drop('_c0').drop('split_cols').drop('colcount')
split_df.show(truncate=60)

+--------+---------------+-----+------+--------------------------------------------------------+
|  folder|       filename|width|height|                                                dog_list|
+--------+---------------+-----+------+--------------------------------------------------------+
|02110627|n02110627_12938|  200|   300|                             [affenpinscher,0,9,173,298]|
|02093754| n02093754_1148|  500|   378|                         [Border_terrier,73,127,341,335]|
|      %s|             %s|  800|   600|                      [Shetland_sheepdog,124,87,576,514]|
|02104029|   n02104029_63|  500|   375|                                    [kuvasz,0,0,499,327]|
|02111500| n02111500_5137|  500|   375|                        [Great_Pyrenees,124,225,403,374]|
|02104365| n02104365_7518|  500|   333|                             [schipperke,146,29,416,309]|
|02105056| n02105056_2834|  500|   375|                             [groenendael,168,0,469,374]|
|02093647|  n02093647_541|  50

# Data validation

## Validating via joins
* Compares data against known values
* Easy to find data in a given set
* Comparatively fast

        parsed_df = spark.read.parquet('parsed_data.parquet')
        company_df = spark.read.parquet('companies.parquet')
        verified_df = parsed_df.join(company_df, parsed_df.company == company_df.company)
        
This automatically removes any rows with a company not in the valid_df 

In [9]:
valid_folders_df = split_df.select('folder').withColumn('folder', F.col('folder').cast('int')).filter(F.col('folder').isNotNull()).distinct()

In [10]:
# Count the number of rows in split_df
split_count = split_df.count()

# Join the DataFrames
joined_df = split_df.join(F.broadcast(valid_folders_df), "folder")

# Compare the number of rows remaining
joined_count = joined_df.count()
print("Before: %d\nAfter: %d" % (split_count, joined_count))

Before: 65
After: 64


## Examining invalid rows

In [11]:
# Determine the row counts for each DataFrame
split_count = split_df.count()
joined_count = joined_df.count()

# Create a DataFrame containing the invalid rows
invalid_df = split_df.join(F.broadcast(joined_df), 'folder', 'left_anti')

# Validate the count of the new DataFrame is as expected
invalid_count = invalid_df.count()
print(" split_df:\t%d\n joined_df:\t%d\n invalid_df: \t%d" % (split_count, joined_count, invalid_count))

# Determine the number of distinct folder rows removed
invalid_folder_count = invalid_df.select('folder').distinct().count()
print("%d distinct invalid folders found" % invalid_folder_count)

 split_df:	65
 joined_df:	64
 invalid_df: 	1
1 distinct invalid folders found


# Final analysis and delivery

## Analysis calculations (UDF)
Calculations using UDF

``` 
def getAvgSale(saleslist):
    totalsales = 0
    count = 0
    for sale in saleslist:
        totalsales += sale[2] + sale[3]
    count += 2
    return totalsales / count

udfGetAvgSale = udf(getAvgSale, DoubleType())
df = df.withColumn('avg_sale', udfGetAvgSale(df.sales_list))
```

## Analysis calculations (inline)
Inline calculations

        df = df.read.csv('datafile')
        df = df.withColumn('avg', (df.total_sales / df.sales_count))
        df = df.withColumn('sq_ft', df.width * df.length)
        df = df.withColumn('total_avg_size', udfComputeTotal(df.entries) / df.numEntries)

## Dog parsing

In [12]:
# Select the dog details and show 10 untruncated rows
joined_df.show(truncate=False)

+--------+---------------+-----+------+--------------------------------------------------------+
|folder  |filename       |width|height|dog_list                                                |
+--------+---------------+-----+------+--------------------------------------------------------+
|02110627|n02110627_12938|200  |300   |[affenpinscher,0,9,173,298]                             |
|02093754|n02093754_1148 |500  |378   |[Border_terrier,73,127,341,335]                         |
|02104029|n02104029_63   |500  |375   |[kuvasz,0,0,499,327]                                    |
|02111500|n02111500_5137 |500  |375   |[Great_Pyrenees,124,225,403,374]                        |
|02104365|n02104365_7518 |500  |333   |[schipperke,146,29,416,309]                             |
|02105056|n02105056_2834 |500  |375   |[groenendael,168,0,469,374]                             |
|02093647|n02093647_541  |500  |333   |[Bedlington_terrier,10,12,462,332]                      |
|02098413|n02098413_1355 |500 

In [13]:
joined_df.printSchema()

root
 |-- folder: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- width: string (nullable = true)
 |-- height: string (nullable = true)
 |-- dog_list: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [14]:
# Define a schema type for the details in the dog list
DogType = StructType([
	StructField("breed", StringType(), False),
    StructField("start_x", IntegerType(), False),
    StructField("start_y", IntegerType(), False),
    StructField("end_x", IntegerType(), False),
    StructField("end_y", IntegerType(), False)
])
DogType

StructType([StructField('breed', StringType(), False), StructField('start_x', IntegerType(), False), StructField('start_y', IntegerType(), False), StructField('end_x', IntegerType(), False), StructField('end_y', IntegerType(), False)])

> If you're wondering why we didn't just define a full schema for the import, the Spark CSV parser is not capable of using complex schema types using lists.

## Per image count

In [15]:
# Create a function to return the number and type of dogs as a tuple
def dogParse(doglist):
    dogs = []
    for dog in doglist:
        (breed, start_x, start_y, end_x, end_y) = dog.split(',')
        dogs.append((breed, int(start_x), int(start_y), int(end_x), int(end_y)))
    return dogs

# Create a UDF
udfDogParse = F.udf(dogParse, ArrayType(DogType))

# Use the UDF to list of dogs and drop the old column
joined_df = joined_df.withColumn('dogs', udfDogParse('dog_list')).drop('dog_list')

In [16]:
# Show the number of dogs in the first 10 rows
joined_df.withColumn('dog_count', F.size('dogs')).show(truncate=False)

+--------+---------------+-----+------+--------------------------------------------------------------------+---------+
|folder  |filename       |width|height|dogs                                                                |dog_count|
+--------+---------------+-----+------+--------------------------------------------------------------------+---------+
|02110627|n02110627_12938|200  |300   |[{affenpinscher, 0, 9, 173, 298}]                                   |1        |
|02093754|n02093754_1148 |500  |378   |[{Border_terrier, 73, 127, 341, 335}]                               |1        |
|02104029|n02104029_63   |500  |375   |[{kuvasz, 0, 0, 499, 327}]                                          |1        |
|02111500|n02111500_5137 |500  |375   |[{Great_Pyrenees, 124, 225, 403, 374}]                              |1        |
|02104365|n02104365_7518 |500  |333   |[{schipperke, 146, 29, 416, 309}]                                   |1        |
|02105056|n02105056_2834 |500  |375   |[{groenen

In [17]:
joined_df.printSchema()

root
 |-- folder: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- width: string (nullable = true)
 |-- height: string (nullable = true)
 |-- dogs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- breed: string (nullable = false)
 |    |    |-- start_x: integer (nullable = false)
 |    |    |-- start_y: integer (nullable = false)
 |    |    |-- end_x: integer (nullable = false)
 |    |    |-- end_y: integer (nullable = false)



In [18]:
joined_df.show(truncate=False)

+--------+---------------+-----+------+--------------------------------------------------------------------+
|folder  |filename       |width|height|dogs                                                                |
+--------+---------------+-----+------+--------------------------------------------------------------------+
|02110627|n02110627_12938|200  |300   |[{affenpinscher, 0, 9, 173, 298}]                                   |
|02093754|n02093754_1148 |500  |378   |[{Border_terrier, 73, 127, 341, 335}]                               |
|02104029|n02104029_63   |500  |375   |[{kuvasz, 0, 0, 499, 327}]                                          |
|02111500|n02111500_5137 |500  |375   |[{Great_Pyrenees, 124, 225, 403, 374}]                              |
|02104365|n02104365_7518 |500  |333   |[{schipperke, 146, 29, 416, 309}]                                   |
|02105056|n02105056_2834 |500  |375   |[{groenendael, 168, 0, 469, 374}]                                   |
|02093647|n02093647

## Percentage dog pixels

In [19]:
# Define a UDF to determine the number of pixels per image
def dogPixelCount(doglist):
    totalpixels = 0
    for dog in doglist:
        totalpixels += (dog[3] - dog[1]) * (dog[4] - dog[2])
    return totalpixels

# Define a UDF for the pixel count
udfDogPixelCount = F.udf(dogPixelCount, IntegerType())
joined_df = joined_df.withColumn('dog_pixels', udfDogPixelCount('dogs'))

# Create a column representing the percentage of pixels
joined_df = joined_df.withColumn('dog_percent', (joined_df.dog_pixels / (joined_df.width * joined_df.height)) * 100)

joined_df.where('dog_percent > 60').show(10)

+--------+---------------+-----+------+--------------------+----------+-----------------+
|  folder|       filename|width|height|                dogs|dog_pixels|      dog_percent|
+--------+---------------+-----+------+--------------------+----------+-----------------+
|02110627|n02110627_12938|  200|   300|[{affenpinscher, ...|     49997|83.32833333333333|
|02104029|   n02104029_63|  500|   375|[{kuvasz, 0, 0, 4...|    163173|          87.0256|
|02105056| n02105056_2834|  500|   375|[{groenendael, 16...|    112574|60.03946666666666|
|02093647|  n02093647_541|  500|   333|[{Bedlington_terr...|    144640|86.87087087087087|
|02098413| n02098413_1355|  500|   375|[{Lhasa, 39, 1, 4...|    171120|           91.264|
|02093859| n02093859_2309|  330|   500|[{Kerry_blue_terr...|    131878|79.92606060606062|
|02109961| n02109961_1017|  475|   500|[{Eskimo_dog, 43,...|    189189|79.65852631578947|
|02108000| n02108000_3491|  600|   450|[{EntleBucher, 30...|    168667|62.46925925925926|
|02085782|